In [9]:
%load_ext autoreload
%autoreload 2
from vilt.modules import ViLTransformerSS
import pytorch_lightning as pl
from vilt.datamodules.multitask_datamodule import MTDataModule
import json
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
s='''
{
  "exp_name": "finetune_nlvr2_randaug",
  "seed": 0,
  "datasets": [
    "nlvr2"
  ],
  "loss_names": {
    "itm": 0,
    "mlm": 0,
    "mpp": 0,
    "vqa": 0,
    "nlvr2": 1,
    "irtr": 0
  },
  "batch_size": 128,
  "train_transform_keys": [
    "pixelbert_randaug"
  ],
  "val_transform_keys": [
    "pixelbert"
  ],
  "image_size": 384,
  "max_image_len": -1,
  "patch_size": 32,
  "draw_false_image": 0,
  "image_only": false,
  "vqav2_label_size": 3129,
  "max_text_len": 40,
  "tokenizer": "bert-base-uncased",
  "vocab_size": 30522,
  "whole_word_masking": false,
  "mlm_prob": 0.15,
  "draw_false_text": 0,
  "vit": "vit_base_patch32_384",
  "hidden_size": 768,
  "num_heads": 12,
  "num_layers": 12,
  "mlp_ratio": 4,
  "drop_rate": 0.1,
  "optim_type": "adamw",
  "learning_rate": 0.0001,
  "weight_decay": 0.01,
  "decay_power": 1,
  "max_epoch": 10,
  "max_steps": null,
  "warmup_steps": 0.1,
  "end_lr": 0,
  "lr_mult": 1,
  "get_recall_metric": false,
  "resume_from": null,
  "fast_dev_run": false,
  "val_check_interval": 1,
  "test_only": false,
  "data_root": "dataset",
  "log_dir": "result",
  "per_gpu_batchsize": 32,
  "num_gpus": 8,
  "num_nodes": 1,
  "load_path": "weights/vilt_200k_mlm_itm.ckpt",
  "num_workers": 8,
  "precision": 16
}
'''

In [11]:
_config = json.loads(s)

In [13]:
model = ViLTransformerSS(_config)

In [14]:
exp_name = f'{_config["exp_name"]}'

os.makedirs(_config["log_dir"], exist_ok=True)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=1,
    verbose=True,
    monitor="val/the_metric",
    mode="max",
    save_last=True,
)
logger = pl.loggers.TensorBoardLogger(
    _config["log_dir"],
    name=f'{exp_name}_seed{_config["seed"]}_from_{_config["load_path"].split("/")[-1][:-5]}',
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval="step")
callbacks = [checkpoint_callback, lr_callback]

num_gpus = (
    _config["num_gpus"]
    if isinstance(_config["num_gpus"], int)
    else len(_config["num_gpus"])
)

grad_steps = _config["batch_size"] // (
    _config["per_gpu_batchsize"] * num_gpus * _config["num_nodes"]
)

max_steps = _config["max_steps"] if _config["max_steps"] is not None else None

trainer = pl.Trainer(
    gpus=0,
    num_nodes=_config["num_nodes"],
    #precision=_config["precision"],
    #accelerator="ddp",
    benchmark=True,
    deterministic=True,
    max_epochs=_config["max_epoch"] if max_steps is None else 1000,
    max_steps=max_steps,
    callbacks=callbacks,
    logger=logger,
    prepare_data_per_node=False,
    replace_sampler_ddp=False,
    accumulate_grad_batches=grad_steps,
    log_every_n_steps=10,
    flush_logs_every_n_steps=10,
    resume_from_checkpoint=_config["resume_from"],
    weights_summary="top",
    fast_dev_run=_config["fast_dev_run"],
    val_check_interval=_config["val_check_interval"],
)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [21]:
dm = MTDataModule(_config, dist=True)
trainer.fit(model, datamodule=dm)

['nlvr2_train']
[pyarrow.Table
image_0: binary
image_1: binary
questions: list<item: string>
  child 0, item: string
answers: list<item: string>
  child 0, item: string
identifier: string]
['nlvr2_dev', 'nlvr2_test1']
[]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/root/anaconda3/envs/vilt/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_43163/2229781289.py", line 2, in <cell line: 2>
    trainer.fit(model, datamodule=dm)
  File "/root/anaconda3/envs/vilt/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 457, in fit
    self.accelerator_backend.setup(model)
  File "/root/anaconda3/envs/vilt/lib/python3.9/site-packages/pytorch_lightning/accelerators/cpu_accelerator.py", line 45, in setup
    self.trainer.call_setup_hook(model)
  File "/root/anaconda3/envs/vilt/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 859, in call_setup_hook
    self.datamodule.setup(stage_name)
  File "/root/anaconda3/envs/vilt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py", line 92, in wrapped_fn
    return fn(*args, **kwargs)
  File "/root/thesis/ViLT/v